In [ ]:
%Truncation on
%AddDeps org.scalanlp breeze_2.12 1.0-RC2 --transitive --trace --verbose
%AddDeps org.scalanlp breeze-natives_2.12 1.0-RC2 --transitive --trace --verbose
%AddDeps org.scalanlp breeze-viz_2.12 1.0-RC2 --transitive --trace --verbose
%ShowTypes on

addapt to importing from jar :D

In [ ]:
import kernel_lib._

In [ ]:
import com.github.fommil.netlib.LAPACK.{getInstance => lapack}

# Linalg UDFs and Utils

In [ ]:
object LinalgUtils extends Serializable {
    def inplace_lower_triangular_solve (
        L:breeze.linalg.DenseMatrix[Double], 
        x: breeze.linalg.DenseVector[Double]
    ):Unit = {
        val N = L.rows
        val info = new  org.netlib.util.intW (0)
        lapack.dtrtrs(
            "L" /* lower triangular */,
            "N",
            "N",
            N /* number of rows */,
            1 /* number of right hand sides */,
            L.data,
            scala.math.max(1, N) /* LDA */,
            x.data,
            scala.math.max(1, N) /* LDB */,
            info
        )
        
        assert(info.`val` >= 0)

        if (info.`val` > 0)
        throw new breeze.linalg.MatrixSingularException
    }
    
    def inplace_lower_triangular_solve (
        L:breeze.linalg.DenseMatrix[Double], 
        X: breeze.linalg.DenseMatrix[Double]
    ):Unit = {
        val N = L.rows
        val info = new  org.netlib.util.intW (0)
        lapack.dtrtrs(
            "L" /* lower triangular */,
            "N",
            "N",
            N /* number of rows */,
            X.cols /* number of right hand sides */,
            L.data,
            scala.math.max(1, N) /* LDA */,
            X.data,
            scala.math.max(1, N) /* LDB */,
            info
        )
        
        assert(info.`val` >= 0)

        if (info.`val` > 0)
        throw new breeze.linalg.NotConvergedException(breeze.linalg.NotConvergedException.Iterations)
    }
    
    def implace_cholesky (X: breeze.linalg.DenseMatrix[Double]): Unit = {
        // set upper triangle to 0
        for (i <- 0 until X.rows; j <- i + 1 until X.cols) {
            X (i, j) = 0.0
        }

        val N = X.rows
        val info = new  org.netlib.util.intW (0)
        lapack.dpotrf(
            "L" /* lower triangular */,
            N /* number of rows */,
            X.data,
            scala.math.max(1, N) /* LDA */,
            info
        )
        // A value of info.`val` < 0 would tell us that the i-th argument
        // of the call to dpotrf was erroneous (where i == |info.`val`|).
        assert(info.`val` >= 0)

        if (info.`val` > 0)
        throw new breeze.linalg.NotConvergedException(breeze.linalg.NotConvergedException.Iterations)
    }
    
    val norm_sqr_udf = org.apache.spark.sql.functions.udf (
        (y:Seq[Double]) => {
            val dense_y = new breeze.linalg.DenseVector (y.toArray)

            dense_y.t * dense_y
        }
    )
    
    val norm_udf = org.apache.spark.sql.functions.udf (
        (y:Seq[Double]) => {
            val dense_y = new breeze.linalg.DenseVector (y.toArray)

            breeze.linalg.norm (dense_y)
        }
    )
    
    val normalize_udf = org.apache.spark.sql.functions.udf (
        (y:Seq[Double]) => {
            val dense_y = new breeze.linalg.DenseVector (y.toArray)

            dense_y /= breeze.linalg.norm (dense_y)
            
            dense_y.data.toSeq
        }
    )
    
    val cap_to_one_udf = org.apache.spark.sql.functions.udf (
        scala.math.min (_:Double, 1.0)
    )
}

# Kernel Pack

## Utils

In [ ]:
object KernelUtils extends Serializable {    
    val kernel_norm = (
        x:breeze.linalg.DenseVector[Double], 
        kernel_func:(breeze.linalg.DenseVector[Double], breeze.linalg.DenseVector[Double]) => Double
    ) => kernel_func (x, x)

    def kernel_cross (
        X:breeze.linalg.DenseMatrix[Double],
        y:breeze.linalg.DenseVector[Double],
        kernel_func:(breeze.linalg.DenseVector[Double], breeze.linalg.DenseVector[Double]) => Double
    ):breeze.linalg.DenseVector[Double] = {
        X(::, breeze.linalg.*).map (
            (x:breeze.linalg.DenseVector[Double]) => kernel_func (x, y)
        ).t
    }

    def kernel_proj (
        X:breeze.linalg.DenseMatrix[Double],
        y:breeze.linalg.DenseVector[Double],
        sqrt_GX:breeze.linalg.DenseMatrix[Double],
        kernel_func:(breeze.linalg.DenseVector[Double], breeze.linalg.DenseVector[Double]) => Double
    ):breeze.linalg.DenseVector[Double] = {
        val y_proj = kernel_cross (X, y, kernel_func)
        
        LinalgUtils.inplace_lower_triangular_solve (sqrt_GX, y_proj)
        
        y_proj
    }
    
    // alters out vector's content
    def buff_kernel_proj (
        X:breeze.linalg.DenseMatrix[Double],
        y:breeze.linalg.DenseVector[Double],
        sqrt_GX:breeze.linalg.DenseMatrix[Double],
        kernel_func:(breeze.linalg.DenseVector[Double], breeze.linalg.DenseVector[Double]) => Double,
        out:breeze.linalg.DenseVector[Double]
    ):Unit = {
        for (i <- 0 until X.cols) {
            out (i) = kernel_func (X(::, i), y)
        }
        
        LinalgUtils.inplace_lower_triangular_solve (sqrt_GX, out)
    }

    def kernel_crosses (
        X:breeze.linalg.DenseMatrix[Double],
        Y:breeze.linalg.DenseMatrix[Double],
        kernel_func:(breeze.linalg.DenseVector[Double], breeze.linalg.DenseVector[Double]) => Double
    ):breeze.linalg.DenseMatrix[Double] = {
        // does Y then X because column major
        val kernel_cross_array = Y(::, breeze.linalg.*).iterator.flatMap (
            (y:breeze.linalg.DenseVector[Double]) => {
                X(::, breeze.linalg.*).iterator.map (
                    (x:breeze.linalg.DenseVector[Double]) => kernel_func (x, y)
                )
            }
        ).toArray

        new breeze.linalg.DenseMatrix (
            X.cols, kernel_cross_array.length / X.cols, 
            kernel_cross_array
        )
    }
    
    def kernel_projs (
        X:breeze.linalg.DenseMatrix[Double],
        Y:breeze.linalg.DenseMatrix[Double],
        sqrt_GX:breeze.linalg.DenseMatrix[Double],
        kernel_func:(breeze.linalg.DenseVector[Double], breeze.linalg.DenseVector[Double]) => Double
    ):breeze.linalg.DenseMatrix[Double] = {
        val Y_proj = kernel_crosses (X, Y, kernel_func)
        
        LinalgUtils.inplace_lower_triangular_solve (sqrt_GX, Y_proj)
        
        Y_proj
    }
    
    def kernel_leverages (
        X:breeze.linalg.DenseMatrix[Double],
        Y:breeze.linalg.DenseMatrix[Double],
        sqrt_GX:breeze.linalg.DenseMatrix[Double],
        kernel_func:(breeze.linalg.DenseVector[Double], breeze.linalg.DenseVector[Double]) => Double
    ):breeze.linalg.DenseVector[Double] = {
        val Y_proj = kernel_projs (X, Y, sqrt_GX, kernel_func)
        
        Y_proj :*= Y_proj
        
        val residual = breeze.linalg.sum (Y_proj.t (breeze.linalg.*, ::))
        
        for (i <- 0 until residual.length) {
            residual (i) = kernel_norm (Y (::, i), kernel_func) - residual (i)
        }
        
        residual
    }

    def kernel_gram (
        X:breeze.linalg.DenseMatrix[Double],
        kernel_func:(breeze.linalg.DenseVector[Double], breeze.linalg.DenseVector[Double]) => Double
    ):breeze.linalg.DenseMatrix[Double] = {
        val gram = new breeze.linalg.DenseMatrix[Double](X.cols, X.cols)
        var k_ij = 0.0
        var Xi:breeze.linalg.DenseVector[Double] = null

        for (i <- 0 until X.cols) {
            Xi = X(::, i)

            for (j <- 0 until i) {
                k_ij = kernel_func (Xi, X(::, j))
                gram(i, j) = k_ij
                gram(j, i) = k_ij
            }

            gram (i, i) = kernel_func (Xi, Xi)
        }

        gram
    }
    
    def inplace_kernel_gram (
        X:breeze.linalg.DenseMatrix[Double],
        kernel_func:(breeze.linalg.DenseVector[Double], breeze.linalg.DenseVector[Double]) => Double,
        out:breeze.linalg.DenseMatrix[Double]
    ):Unit = {
        assert (
            X.cols == out.cols && X.cols == out.rows, 
            """Output Matrix did not match input X dimension
              |Output dims ${out.rows} x ${out.cols}
              |X dim ${X.cols}"""
        )
        
        var k_ij = 0.0
        var Xi:breeze.linalg.DenseVector[Double] = null

        for (i <- 0 until X.cols) {
            Xi = X(::, i)

            for (j <- 0 until i) {
                k_ij = kernel_func (Xi, X(::, j))
                out(i, j) = k_ij
                out(j, i) = k_ij
            }

            out (i, i) = kernel_func (Xi, Xi)
        }
    }
    
    def eval_chol (
        X:breeze.linalg.DenseMatrix[Double], 
        regularizer_diag:breeze.linalg.DenseVector[Double], 
        lambda:Double,
        kernel_func:(breeze.linalg.DenseVector[Double], breeze.linalg.DenseVector[Double]) => Double
    ):breeze.linalg.DenseMatrix[Double] = {
        val eventual_out_mat = kernel_gram (X, kernel_func)

        // slightly inefficient but garantees a pure function
        eventual_out_mat += breeze.linalg.diag (lambda * regularizer_diag)
        
        //cholesky
        LinalgUtils.implace_cholesky (eventual_out_mat)
        
        eventual_out_mat
    }
    
    def implace_eval_chol (
        X:breeze.linalg.DenseMatrix[Double], 
        regularizer_diag:breeze.linalg.DenseVector[Double], 
        lambda:Double,
        kernel_func:(breeze.linalg.DenseVector[Double], breeze.linalg.DenseVector[Double]) => Double,
        eventual_out_mat:breeze.linalg.DenseMatrix[Double]
    ):Unit = {
        inplace_kernel_gram (X, kernel_func, eventual_out_mat)

        // slightly inefficient but garantees a pure function
        eventual_out_mat += breeze.linalg.diag (lambda * regularizer_diag)
        
        //cholesky
        LinalgUtils.implace_cholesky (eventual_out_mat)  
    }
}

## Pack

In [ ]:
object KernelPack {
    def apply (
        X:breeze.linalg.DenseMatrix[Double],
        regularizer_diag:breeze.linalg.DenseVector[Double], 
        lambda:Double,
        kernel_func:(
            breeze.linalg.DenseVector[Double], 
            breeze.linalg.DenseVector[Double]
        ) => Double
    ) = {
        val kpack = new KernelPack (
            kernel_func
        )

        kpack.update (X, regularizer_diag, lambda)

        kpack
    }

    
    def knorm_factory (
        kernel_func:(
            breeze.linalg.DenseVector[Double], 
            breeze.linalg.DenseVector[Double]
        ) => Double
    ) = {
        org.apache.spark.sql.functions.udf (
            (y:Seq[Double]) => KernelUtils.kernel_norm (
                new breeze.linalg.DenseVector (y.toArray),
                kernel_func
            )
        )
    }
    
    def kproj_factory (
        X:breeze.linalg.DenseMatrix[Double],
        sqrt_GX:breeze.linalg.DenseMatrix[Double],
        kernel_func:(
            breeze.linalg.DenseVector[Double], 
            breeze.linalg.DenseVector[Double]
        ) => Double
    ) = {
        org.apache.spark.sql.functions.udf (
            (y:Seq[Double]) => KernelUtils.kernel_proj (
                X, new breeze.linalg.DenseVector (y.toArray), sqrt_GX, kernel_func
            ).data.toSeq
        )
    }
    
    def kleverage_factory (
        X:breeze.linalg.DenseMatrix[Double],
        sqrt_GX:breeze.linalg.DenseMatrix[Double],
        kernel_func:(
            breeze.linalg.DenseVector[Double], 
            breeze.linalg.DenseVector[Double]
        ) => Double
    ) = {
        org.apache.spark.sql.functions.udf (
            (y:Seq[Double]) => {
                val dense_y = new breeze.linalg.DenseVector (y.toArray)
                val y_proj = KernelUtils.kernel_proj (
                    X, dense_y, sqrt_GX, kernel_func
                )

                KernelUtils.kernel_norm (
                    dense_y, kernel_func
                ) - (y_proj.t * y_proj)
            }:Double
        )
    }
}

class KernelPack private (
    val kernel_func:(
        breeze.linalg.DenseVector[Double], 
        breeze.linalg.DenseVector[Double]
    ) => Double
) extends Serializable {
    private var _kproj_udf:org.apache.spark.sql.expressions.UserDefinedFunction = null
    private var _kleverage_udf:org.apache.spark.sql.expressions.UserDefinedFunction = null
    
    def kproj_udf () = _kproj_udf
    def kleverage_udf () = _kleverage_udf

    val knorm_udf = KernelPack.knorm_factory (kernel_func)

    def update (
        X:breeze.linalg.DenseMatrix[Double],
        regularizer_diag:breeze.linalg.DenseVector[Double], 
        lambda:Double
    ):Unit = {
        val sqrt_GX = KernelUtils.eval_chol (
            X, regularizer_diag, 
            lambda, this.kernel_func
        )
        
        this._kproj_udf = KernelPack.kproj_factory (X, sqrt_GX, this.kernel_func)
        this._kleverage_udf = KernelPack.kleverage_factory (X, sqrt_GX, this.kernel_func)
    }
}

## Spaces

In [ ]:
object KernelSpaces {
    def apply (
        X:breeze.linalg.DenseMatrix[Double],
        regularizer_diag:breeze.linalg.DenseVector[Double], 
        X_ids:Seq[Long],
        lambda:Double,
        kernel_func:(
            breeze.linalg.DenseVector[Double], 
            breeze.linalg.DenseVector[Double]
        ) => Double,
        Xs_max_count:Int
    ) = {
        val kspaces = new KernelSpaces (
            new Array[breeze.linalg.DenseMatrix[Double]] (Xs_max_count), // Xs
            new Array[breeze.linalg.DenseVector[Double]] (Xs_max_count), // regularizer_diags
            new Array[breeze.linalg.DenseMatrix[Double]] (Xs_max_count), // sqrt_GXs
            new Array[Seq[Long]] (Xs_max_count), // Xs_ids
            kernel_func
        )

        kspaces.update_lambda (lambda)

        kspaces.append (
            X,
            regularizer_diag,
            X_ids
        )
        
        kspaces
    }
    
    def knorm_factory (
        kernel_func:(
            breeze.linalg.DenseVector[Double], 
            breeze.linalg.DenseVector[Double]
        ) => Double
    ) = {
        org.apache.spark.sql.functions.udf (
            (y:Seq[Double]) => KernelUtils.kernel_norm (
                new breeze.linalg.DenseVector (y.toArray),
                kernel_func
            )
        )
    }
    
    def kproj_factory (
        Xs:Seq[breeze.linalg.DenseMatrix[Double]],
        sqrt_GXs:Seq[breeze.linalg.DenseMatrix[Double]],
        kernel_func:(
            breeze.linalg.DenseVector[Double], 
            breeze.linalg.DenseVector[Double]
        ) => Double
    ) = {
        val proj_len = Xs.map (_.cols).sum
        val kspace_pairs = Xs.zip (sqrt_GXs)

        org.apache.spark.sql.functions.udf (
            (y:Seq[Double]) => {
                val dense_y = new breeze.linalg.DenseVector (y.toArray)
                val y_proj = new breeze.linalg.DenseVector[Double] (proj_len) 

                var offset:Int = 0

                // aparently pattern matching requires lowercase...
                for ((aX, sqrt_GX) <- kspace_pairs) {
                    KernelUtils.buff_kernel_proj (
                        aX, dense_y, sqrt_GX, kernel_func,
                        y_proj(offset until offset + aX.cols)
                    )
                }

                y_proj.data.toSeq
            }
        )
    }
    
    // leverage is evaluated via 
    // knorm - sum (leverages w.r.t. each X)
    // may turn out negative
    // there may be alternatives
    def pseudo_kleverage_factory (
        Xs:Seq[breeze.linalg.DenseMatrix[Double]],
        sqrt_GXs:Seq[breeze.linalg.DenseMatrix[Double]],
        kernel_func:(
            breeze.linalg.DenseVector[Double], 
            breeze.linalg.DenseVector[Double]
        ) => Double
    ) = {
        val buffer_len = Xs.map (_.cols).max
        val kspace_pairs = Xs.zip (sqrt_GXs)

        org.apache.spark.sql.functions.udf (
            (y:Seq[Double]) => {
                val dense_y = new breeze.linalg.DenseVector (y.toArray)
                val buff_y_proj = new breeze.linalg.DenseVector[Double] (buffer_len)

                var residual = KernelUtils.kernel_norm (
                    dense_y, kernel_func
                )

                // aparently pattern matching requires lowercase...
                for ((aX, sqrt_GX) <- kspace_pairs) {
                    KernelUtils.buff_kernel_proj (
                        aX, dense_y, sqrt_GX, kernel_func,
                        buff_y_proj(0 until aX.cols)
                    )
                    residual -= buff_y_proj(0 until aX.cols).t * buff_y_proj(0 until aX.cols)
                }

                residual
            }:Double
        )
    }

    def min_kleverage_factory (
        Xs:Seq[breeze.linalg.DenseMatrix[Double]],
        sqrt_GXs:Seq[breeze.linalg.DenseMatrix[Double]],
        kernel_func:(
            breeze.linalg.DenseVector[Double], 
            breeze.linalg.DenseVector[Double]
        ) => Double
    ) = {
        val buffer_len = Xs.map (_.cols).max
        val kspace_pairs = Xs.zip (sqrt_GXs)

        org.apache.spark.sql.functions.udf (
            (y:Seq[Double]) => {
                val dense_y = new breeze.linalg.DenseVector (y.toArray)
                val buff_y_proj = new breeze.linalg.DenseVector[Double] (buffer_len)

                val norm = KernelUtils.kernel_norm (
                    dense_y, kernel_func
                )

                // aparently pattern matching requires lowercase...
                kspace_pairs.map {
                    case (aX, sqrt_GX) => {
                        KernelUtils.buff_kernel_proj (
                            aX, dense_y, sqrt_GX, kernel_func,
                            buff_y_proj(0 until aX.cols)
                        )
                        norm - buff_y_proj(0 until aX.cols).t * buff_y_proj(0 until aX.cols)
                    }
                }.min
            }:Double
        )
    }
}

class KernelSpaces private (
    private val _Xs:Array[breeze.linalg.DenseMatrix[Double]],
    private val _regularizer_diags:Array[breeze.linalg.DenseVector[Double]],
    private val _sqrt_GXs:Array[breeze.linalg.DenseMatrix[Double]],
    private val _Xs_ids:Array[Seq[Long]],
    val kernel_func:(
        breeze.linalg.DenseVector[Double], 
        breeze.linalg.DenseVector[Double]
    ) => Double
) extends Serializable {
    private var _kproj_udf:org.apache.spark.sql.expressions.UserDefinedFunction = null
    private var _pseudo_kleverage_udf:org.apache.spark.sql.expressions.UserDefinedFunction = null
    private var _min_kleverage_udf:org.apache.spark.sql.expressions.UserDefinedFunction = null
    
    private var _Xs_count:Int = 0
    private var _Xs_last_id:Int = 0
    
    // can be updated, updating all subspaces
    private var _lambda:Double = 0

    val Xs_length = this._Xs.length

    def kproj_udf () = this._kproj_udf
    def pseudo_kleverage_udf () = this._pseudo_kleverage_udf
    def min_kleverage_udf () = this._min_kleverage_udf

    def lambda () = this._lambda
    def Xs_count () = this._Xs_count
    def Xs_last_id () = this._Xs_last_id

    def Xs () = this._Xs.slice (0, this._Xs_count).toSeq
    def sqrt_GXs () = this._sqrt_GXs.slice (0, this._Xs_count).toSeq
    def regularizer_diags () = this._regularizer_diags.slice (0, this._Xs_count).toSeq
    def Xs_ids () = this._Xs_ids.slice (0, this._Xs_count).toSeq

    private def update_udfs ():Unit = {
        // reconstructs udfs
        this._kproj_udf = KernelSpaces.kproj_factory (
            this._Xs.slice (0, this._Xs_count).toSeq, 
            this._sqrt_GXs.slice (0, this._Xs_count).toSeq, 
            this.kernel_func
        )
        
        this._pseudo_kleverage_udf = KernelSpaces.pseudo_kleverage_factory (
            this._Xs.slice (0, this._Xs_count).toSeq, 
            this._sqrt_GXs.slice (0, this._Xs_count).toSeq, 
            this.kernel_func
        )

        this._min_kleverage_udf = KernelSpaces.min_kleverage_factory (
            this._Xs.slice (0, this._Xs_count).toSeq, 
            this._sqrt_GXs.slice (0, this._Xs_count).toSeq, 
            this.kernel_func
        )
    }

    def append (
        X:breeze.linalg.DenseMatrix[Double],
        regularizer_diag:breeze.linalg.DenseVector[Double], 
        X_ids:Seq[Long]
    ):Unit = {
        assert (this.Xs_length > this._Xs_count, "Cannot append anymore matrices, Array full")

        // adding new matrices and vectors
        this._Xs (this._Xs_count) = X
        this._regularizer_diags (this._Xs_count) = regularizer_diag
        this._Xs_ids (this._Xs_count) = X_ids
        
        this._sqrt_GXs (this._Xs_count) = KernelUtils.eval_chol (
            X, regularizer_diag, 
            this._lambda, this.kernel_func
        )

        // updates count
        this._Xs_count += 1
        this._Xs_last_id += 1

        this.update_udfs
    }

    // overwrites matrices when comming back around the ids
    def append_circular (
        X:breeze.linalg.DenseMatrix[Double],
        regularizer_diag:breeze.linalg.DenseVector[Double], 
        X_ids:Seq[Long]        
    ):Unit = {
        // caps out last id if necessary
        // helps with not needing to do it anywhere else
        if (this._Xs_last_id >= this.Xs_length) {
            this._Xs_last_id = 0
        }
        
        // adding new matrices and vectors
        this._Xs (this._Xs_last_id) = X
        this._regularizer_diags (this._Xs_last_id) = regularizer_diag
        this._Xs_ids (this._Xs_count) = X_ids
        
        this._sqrt_GXs (this._Xs_last_id) = KernelUtils.eval_chol (
            X, regularizer_diag, 
            this._lambda, this.kernel_func
        )

        // updates count and last id
        this._Xs_last_id += 1
        
        if (this._Xs_count < this.Xs_length) {
            this._Xs_count += 1
        }

        this.update_udfs
    }
    
    def remove_id (i:Int) {
        assert (i < this._Xs_count && i >= 0, f"Index ${i} out of interval [0, ${this._Xs_count})")
        
        // sets entries to null, because gc (?)
        this._Xs (i) = null
        this._sqrt_GXs (i) = null
        this._regularizer_diags (i) = null
        this._Xs_ids (i) = null
        
        // decrements length
        this._Xs_count -= 1
        
        // sets i to the highest index's (this._Xs_count) values
        this._Xs (i) = this._Xs (this._Xs_count)
        this._sqrt_GXs (i) = this._sqrt_GXs (this._Xs_count)
        this._regularizer_diags (i) = this._regularizer_diags (this._Xs_count)
        this._Xs_ids (i) = this._Xs_ids (this._Xs_count)
        
        this.update_udfs
    }

    // updates all subspaces to conform to new regularization
    // up to understanding how to update without reevaluating cholesky factors
    def update_lambda (
        lambda:Double
    ):Unit = {
        this._lambda = lambda

        // less memory leaky
        for (
            ((aX, sqrt_GX), regularizer_diag) <- this._Xs.slice (
                0, this._Xs_count
            ).zip (
                this._sqrt_GXs
            ).zip (this._regularizer_diags)
        ) {
            KernelUtils.implace_eval_chol (
                aX, regularizer_diag, 
                this._lambda, this.kernel_func,
                sqrt_GX
            )
        }
    }
}

## kernel func

In [ ]:
def kernel_func (
    x:breeze.linalg.DenseVector[Double], 
    y:breeze.linalg.DenseVector[Double]
):Double = x.t * y

# Spark Digits Load

In [ ]:
import org.apache.spark.{sql => s_sql}

In [ ]:
val max_per_P = 2000

In [ ]:
val path = "hdfs://thesis-tiny-python3-anaconda-m/user/pedro.schmidt/digits.snappy.parquet"
//val path = "digits.snappy.parquet"

var df_digits_raw = spark.read.format (
    "parquet"
).load (path).withColumn (
    "features", $"features".cast ("ARRAY<DOUBLE>")
).withColumn (
    "id", s_sql.functions.monotonically_increasing_id
)

df_digits_raw = df_digits_raw.repartition (df_digits_raw.count ().toInt / max_per_P).persist

In [ ]:
df_digits_raw.rdd.getNumPartitions

# Nystrom algorithm

## Features transform

In [ ]:
val image_dim = scala.math.sqrt (
    df_digits_raw.select (
        s_sql.functions.size ($"features")
    ).map (_.getInt (0)).take (1)(0)
).toInt

In [ ]:
// window is centered

val window_x = 7
val window_y = 7
val skip_x = 3
val skip_y = 3

def max_pool (
    I:breeze.linalg.DenseMatrix[Double]
):breeze.linalg.DenseMatrix[Double] = {
    // column major implies image will be
    val I_pooled = new breeze.linalg.DenseMatrix[Double](
        (I.rows - window_y) / skip_y, 
        (I.cols - window_x) / skip_x
    )
    
    for (
        (i, im, ip) <- (0 until I_pooled.rows).map (
            (v:Int) => (v, skip_y * v, skip_y * v + window_y)
        );
        (j, jm, jp) <- (0 until I_pooled.cols).map (
            (v:Int) => (v, skip_x * v, skip_x * v + window_x)
        )
    ) {
        I_pooled (i, j) = breeze.linalg.max (I (im until ip, jm until jp))
    }
    
    I_pooled
}

val max_pool_udf = s_sql.functions.udf (
    (x:Seq[Double]) => {
        max_pool (
            new breeze.linalg.DenseMatrix (image_dim, image_dim, x.toArray)
        ).data.toSeq
    }
)

In [ ]:
val df_data = df_digits_raw.select (
    LinalgUtils.normalize_udf (
        $"features"
//         max_pool_udf ($"features")
    ).as ("nfeatures"), $"label", $"id"
).persist

In [ ]:
val n = df_data.count ().toInt
val rows = df_data.select (
    s_sql.functions.size ($"nfeatures")
).map (_.getInt (0)).take (1)(0)

## Nystrom Loop

val kappa = 1.0
val t = .3
val base_lamb = kappa * kappa / scala.math.min (t, 1.0)
val q = 1.5
// target lambda is 1 / sqrt (n)
val H = (scala.math.log (base_lamb * scala.math.sqrt (n)) / scala.math.log (q)).toInt
val delta = .9

val q2 = 54 * (kappa * kappa) * ((2 + t * t) / (t * t)) * scala.math.log  (12 * H * n / delta)

val max_rows = 5000

val base_beta = scala.math.min (q2 * kappa * kappa / (base_lamb * n), 1.0)

var lambda_h = base_lamb
var beta_h = base_beta

//step 0, i.e. initialization step
var df_sample_set_h:org.apache.spark.sql.Dataset[
    org.apache.spark.sql.Row
] = df_data.select ("nfeatures").sample (beta_h).withColumn (
    "ph_cap",
    LinalgUtils.cap_to_one_udf (
        // first round does not have a kpack
        // with most kernels, kinda meaningless...
        KernelPack.knorm_factory (kernel_func (_,_))($"nfeatures").multiply (q2)
    )
).filter (
    s_sql.functions.rand ().multiply (beta_h) <= $"ph_cap"
).orderBy ("ph_cap").limit (max_rows)

// sample length
// runs most of the computation, lol
var cols = df_sample_set_h.count ().toInt

val kpack_h = KernelPack (
    // X
    new breeze.linalg.DenseMatrix (
        rows, cols, 
        df_sample_set_h.select ("nfeatures").flatMap {
            case row: s_sql.Row => row.getSeq[Double](0)
        }.collect ().toArray
    ), 
    // A
    new breeze.linalg.DenseVector (
        df_sample_set_h.select ("ph_cap").map {
            case row: s_sql.Row => row.getDouble(0)
        }.collect ().toArray
    ), 
    // lambda * n
    lambda_h * n, 
    kernel_func (_,_)
)

lambda_h = base_lamb
beta_h = base_beta

for (i <- 1 until H) {
    println ("----------------------------------------------")
    println (f"Step: ${i}")
    println ("----------------------------------------------")
    // step i
    // lambda and beta updates
    lambda_h /= q
    beta_h = scala.math.min (q2 * kappa * kappa / (lambda_h * n), 1.0)
    
    println (f"lambda_h: ${lambda_h}, beta_h: ${beta_h}")
    println ("----------------------------------------------")
    
    // new sample
    df_sample_set_h = df_data.select ("nfeatures").sample (beta_h).withColumn (
        "ph_cap",
        LinalgUtils.cap_to_one_udf (kpack_h.kleverage_udf ($"nfeatures").multiply (q2))
    ).filter (
        s_sql.functions.rand ().multiply (beta_h) <= $"ph_cap"
    ).orderBy (s_sql.functions.negate ($"ph_cap")).limit (max_rows)

    // sample length
    // runs most of the computation, lol
    cols = df_sample_set_h.count ().toInt

    // kernel udfs update
    kpack_h.update (
        // X
        new breeze.linalg.DenseMatrix (
            rows, cols, 
            df_sample_set_h.select ("nfeatures").flatMap {
                case row: s_sql.Row => row.getSeq[Double](0)
            }.collect ().toArray
        ), 
        // A
        new breeze.linalg.DenseVector (
            df_sample_set_h.select ("ph_cap").map {
                case row: s_sql.Row => row.getDouble(0)
            }.collect ().toArray
        ), 
        // lambda * n
        lambda_h * n
    )
    
    println (f"cols: ${cols}")
    println ("----------------------------------------------")
    
    // global leverage score
    df_data.select (
        kpack_h.kleverage_udf ($"nfeatures").as ("ph")
    ).agg (s_sql.functions.sum ($"ph")).show
}

## Nystrom spaces loop

In [ ]:
val kappa = 1.0
val t = .3
val base_lamb = kappa * kappa / scala.math.min (t, 1.0)
val q = 2
// target lambda is 1 / sqrt (n)
val H = (scala.math.log (base_lamb * scala.math.sqrt (n)) / scala.math.log (q)).toInt
val delta = .9

// val q2 = 54 * (kappa * kappa) * ((2 + t * t) / (t * t)) * scala.math.log  (12 * H * n / delta)
val q2 = (kappa * kappa) * ((2 + t * t) / (t * t)) * scala.math.log  (12 * H * n / delta)

val max_rows = 1000
val max_spaces = 4
val max_non_improv = 10

val base_beta = scala.math.min (q2 * kappa * kappa / (base_lamb), 1.0)

In [ ]:
var lambda_h = base_lamb
var beta_h = base_beta

//step 0, i.e. initialization step
var df_sample_set_h:org.apache.spark.sql.Dataset[
    org.apache.spark.sql.Row
] = df_data.select ($"nfeatures", $"id").sample (beta_h).withColumn (
    "ph_cap",
    LinalgUtils.cap_to_one_udf (
        // first round does not have a kpack
        // with most kernels, kinda meaningless...
        KernelPack.knorm_factory (kernel_func (_,_))($"nfeatures").multiply (scala.math.log  (12 * H * n / delta))
    )
).filter (
    s_sql.functions.rand () <= LinalgUtils.cap_to_one_udf ($"ph_cap")
).orderBy (s_sql.functions.negate ($"ph_cap")).limit (max_rows).persist

// sample length
var cols = df_sample_set_h.count ().toInt

val kspaces_h = KernelSpaces (
    // X
    new breeze.linalg.DenseMatrix (
        rows, cols, 
        df_sample_set_h.select ("nfeatures").flatMap {
            case row: s_sql.Row => row.getSeq[Double](0)
        }.collect ().toArray
    ), 
    // A
    new breeze.linalg.DenseVector (
        df_sample_set_h.select (
            LinalgUtils.cap_to_one_udf ($"ph_cap")
        ).map {
            case row: s_sql.Row => row.getDouble(0)
        }.collect ().toArray
    ),
    // X_ids
    df_sample_set_h.select (
        $"id"
    ).map {
        case row: s_sql.Row => row.getLong(0)
    }.collect (),
    // lambda * n
    lambda_h,// * n, 
    kernel_func (_,_),
    max_spaces
)

df_sample_set_h = df_sample_set_h.unpersist (true)

In [ ]:
var non_improv_count = 0
var min_leverage:Double = df_data.agg (
    s_sql.functions.sum (kspaces_h.min_kleverage_udf () ($"nfeatures").as ("best_ph"))
).map {
    case row: s_sql.Row => row.getDouble(0)
}.collect ()(0)

var current_leverage:Double = 0.0

var best_Xs = kspaces_h.Xs
var best_sqrt_GXs = kspaces_h.sqrt_GXs
var best_regularizer_diags = kspaces_h.regularizer_diags
var best_ids = kspaces_h.Xs_ids

var i = 1

In [ ]:
i = 1
lambda_h = base_lamb

retain best current set of values

In [ ]:
while (non_improv_count < max_non_improv) {
    println ("----------------------------------------------")
    println (f"i: ${i}")
    println ("----------------------------------------------")
    
    // step i
    // lambda and beta updates
    
    if (i < H) {
        lambda_h /= q
    }

    beta_h = scala.math.min (q2 * kappa * kappa / (lambda_h), 1.0)
    
    println (f"lambda_h: ${lambda_h}, beta_h: ${beta_h}")
    println ("----------------------------------------------")
    
    // removal of subspaces when array gets full
    if (kspaces_h.Xs_length <= i) {
        println (f"removed a X at i: ${(i - 1) % (kspaces_h.Xs_length - 1)}")
        println ("----------------------------------------------")
        kspaces_h.remove_id ((i - 1) % (kspaces_h.Xs_length - 1))
    }
    
    kspaces_h.update_lambda (
        // lambda * n
        lambda_h// * n
    )
    
    // new sample
    df_sample_set_h = df_data.select ($"nfeatures", $"id").sample (beta_h).withColumn (
        "ph_cap",
        kspaces_h.min_kleverage_udf () ($"nfeatures").multiply (scala.math.log  (12 * H * n / delta))
    ).filter (
        s_sql.functions.rand () <= LinalgUtils.cap_to_one_udf ($"ph_cap")
    ).orderBy (s_sql.functions.negate ($"ph_cap")).limit (max_rows).persist

    // sample length
    cols = df_sample_set_h.count ().toInt

    // kernel udfs update
    kspaces_h.append (
        // X
        new breeze.linalg.DenseMatrix (
            rows, cols, 
            df_sample_set_h.select ("nfeatures").flatMap {
                case row: s_sql.Row => row.getSeq[Double](0)
            }.collect ().toArray
        ), 
        // A
        new breeze.linalg.DenseVector (
            df_sample_set_h.select (
                LinalgUtils.cap_to_one_udf ($"ph_cap")
            ).map {
                case row: s_sql.Row => row.getDouble(0)
            }.collect ().toArray
        ),
        // X_ids
        df_sample_set_h.select (
            $"id"
        ).map {
            case row: s_sql.Row => row.getLong(0)
        }.collect ()
    )
    
    println (f"cols: ${cols}")
    println ("----------------------------------------------")
    
    df_sample_set_h = df_sample_set_h.unpersist (true)
    
    println ("Cross leverages between the Xis")
    for (
        k <- 0 until kspaces_h.Xs_count
    ) {
        print (k)
        print ("    ")

        for (l <- 0 until kspaces_h.Xs_count) {
            print (f"""${
                breeze.linalg.sum (KernelUtils.kernel_leverages (
                    kspaces_h.Xs ()(k),
                    kspaces_h.Xs ()(l),
                    kspaces_h.sqrt_GXs ()(k),
                    kspaces_h.kernel_func
                ))
            }%.2f""")
            print (" | ")
        }

        print ("\n")
    }
    
    println ("----------------------------------------------")
    
    current_leverage = df_data.agg (
        s_sql.functions.sum (kspaces_h.min_kleverage_udf () ($"nfeatures").as ("best_ph"))
    ).map {
        case row: s_sql.Row => row.getDouble(0)
    }.collect ()(0)
    
    println (f"current leverage sum: ${current_leverage}, current best ${min_leverage}")
    println ("----------------------------------------------")
    
    if (current_leverage < min_leverage) {
        println (f"improvement happend!")
        println ("----------------------------------------------")
        min_leverage = current_leverage
        // makes so that it stops when 
        // no improvement happens
        // max_non_improv times
        non_improv_count -= 1
        best_Xs = kspaces_h.Xs
        best_sqrt_GXs = kspaces_h.sqrt_GXs
        best_regularizer_diags = kspaces_h.regularizer_diags
        best_ids = kspaces_h.Xs_ids
    }
    else {
        non_improv_count += 1
        println (f"current non-improvement: ${non_improv_count}")
        println ("----------------------------------------------")
    }
    
    i += 1
}

make evalutaion for best spaces

In [ ]:
best_Xs.foreach {
    case aX => println (aX.cols)
}

In [30]:
for (i <- 0 until best_Xs.length) {
    print (i)
    print ("    ")

    for (j <- 0 until best_Xs.length) {
        print (f"""${
            breeze.linalg.sum (KernelUtils.kernel_leverages (
                best_Xs (i),
                best_Xs (j),
                best_sqrt_GXs (i),
                kspaces_h.kernel_func
            ))
        }%.2f""")
        print (" | ")
    }

    print ("\n")
}

4.53 | 3.06 | 0.66 | 


evaluate leverage scores w.r.t each subspace

evaluate 'linear independence' between the subspaces